In [ ]:
    import requests
    import pandas as pd
    from datetime import date

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()


    # Step 4: ATR Function
    def ATR(df, n):
        df = df.copy()
        df["H-L"] = abs(df["high"] - df["low"])
        df["H-PC"] = abs(df["high"] - df["close"].shift(1))
        df["L-PC"] = abs(df["low"] - df["close"].shift(1))
        df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1, skipna=False)
        df["ATR"] = df["TR"].ewm(span=n, adjust=False, min_periods=n).mean()
        df = df.drop(["H-L", "H-PC", "L-PC", "TR"], axis=1)
        return df
        

    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        AtrValue =ATR(symbol_data, 14)
        print(AtrValue.tail())
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-06-13 15:05:00+05:30  24702.60  24727.90  24702.60  24727.90       0  0   
2025-06-13 15:10:00+05:30  24727.95  24728.45  24717.20  24725.60       0  0   
2025-06-13 15:15:00+05:30  24725.15  24725.90  24709.75  24717.95       0  0   
2025-06-13 15:20:00+05:30  24718.70  24733.15  24718.10  24729.35       0  0   
2025-06-13 15:25:00+05:30  24728.55  24744.95  24728.45  24735.55       0  0   

                                 ATR  
Datetime                              
2025-06-13 15:05:00+05:30  21.454123  
2025-06-13 15:10:00+05:30  20.093574  
2025-06-13 15:15:00+05:30  19.567764  
2025-06-13 15:20:00+05:30  18.985395  
2025-06-13 15:25:00+05:30  18.654009  
